In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 41.4 MB/s eta 0:00:00


In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4296185 sha256=3ef983b703bb8a64ac81462f222c5492ded792bcec5dcb2d5cb151c2c90e1ee8
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [ ]:
import fasttext
import numpy as np

In [ ]:
word_pairs=[]
with open('/content/drive/MyDrive/sarvam_project/trunc_lexicons.txt') as f:
  for line in f.readlines():
    x,y=line.strip().split()
    word_pairs.append([x,y])


In [ ]:
hi_matrix=np.load('/content/drive/MyDrive/sarvam_project/hi_embeddings.npy')
en_matrix=np.load('/content/drive/MyDrive/sarvam_project/en_embeddings.npy')

In [ ]:
list_en_words=[]
with open('/content/drive/MyDrive/sarvam_project/list_en_words.txt') as f:
  for line in f.readlines():
    list_en_words.append(line.strip())

In [ ]:
list_hi_words=[]
with open('/content/drive/MyDrive/sarvam_project/list_hi_words.txt') as f:
  for line in f.readlines():
    list_hi_words.append(line.strip())

In [ ]:
en_aligned=np.load('/content/drive/MyDrive/sarvam_project/en_aligned.npy')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Function to find the nearest neighbor based on cosine similarity
def find_nearest_neighbor(embedding, target_matrix):
    similarities = cosine_similarity([embedding], target_matrix)[0]
    nearest_neighbor_idx = np.argmax(similarities)
    return nearest_neighbor_idx, similarities[nearest_neighbor_idx]

# Verify alignment quality by comparing aligned English words with Hindi translations
correct_matches = 0
total_pairs = hi_matrix.shape[0]

for i in range(total_pairs):
    index_find=list_en_words.index(word_pairs[i][1])
    aligned_embedding = en_aligned[index_find]
    expected_hindi_embedding = hi_matrix[i]
    #print(aligned_embedding.shape,expected_hindi_embedding.shape)

    # Find the nearest neighbor for the aligned English word in the Hindi embedding space
    nearest_neighbor_idx, similarity_score = find_nearest_neighbor(aligned_embedding, hi_matrix)

    # Compare the found nearest neighbor with the expected Hindi word
    if (nearest_neighbor_idx==i):
        correct_matches += 1

# Compute accuracy
accuracy = correct_matches / total_pairs
print(f"Alignment Accuracy while training: {accuracy * 100:.2f}%")


KeyboardInterrupt: 

In [ ]:
all_hi_matrix=np.load('/content/drive/MyDrive/sarvam_project/all_hni_matrix.npy')

In [ ]:
with open('/content/drive/MyDrive/sarvam_project/word_translation_test.txt') as f:
    test_word_pairs = [line.strip().split() for line in f.readlines()]

In [ ]:
# import numpy as np
# import faiss  # Facebook's ANN library for efficient similarity search

# # Preprocess: Build an FAISS index for the Hindi matrix
# d = all_hi_matrix.shape[1]  # Dimensionality of the embeddings
# index = faiss.IndexFlatIP(d)  # Inner Product (for cosine similarity) index
# faiss.normalize_L2(all_hi_matrix)  # Normalize vectors for cosine similarity
# index.add(all_hi_matrix)  # Add the Hindi embeddings to the index

# correct_matches_at_5 = 0
# valid_pairs = [
#     pair for pair in test_word_pairs
#     if pair[1] in list_en_words and pair[0] in list_hi_words
# ]

# for pair in valid_pairs:
#     en_word, hi_word = pair[1], pair[0]
#     en_idx = list_en_words.index(en_word)
#     hi_idx = list_hi_words.index(hi_word)

#     # Normalize the aligned embedding for cosine similarity
#     aligned_embedding = en_aligned[en_idx].reshape(1, -1)
#     faiss.normalize_L2(aligned_embedding)

#     # Search for the k nearest neighbors
#     _, nearest_neighbors_idx = index.search(aligned_embedding, 5)

#     # Check if the expected Hindi index is among the neighbors
#     if hi_idx in nearest_neighbors_idx[0]:
#         correct_matches_at_5 += 1

# # Calculate precision@k
# precision_at_5 = correct_matches_at_5 / len(valid_pairs)
# print(f"Precision@k (5): {precision_at_5 * 100:.2f}%")


In [ ]:
import numpy as np
import faiss  # Facebook's ANN library for efficient similarity search

def calculate_precision_at_k(
    k, all_hi_matrix, en_aligned, test_word_pairs, list_en_words, list_hi_words, index=None
):
    """
    Calculate precision@k for word embeddings using FAISS.

    Parameters:
        k (int): Number of nearest neighbors to consider.
        all_hi_matrix (np.ndarray): Hindi word embeddings.
        en_aligned (np.ndarray): Aligned English word embeddings.
        test_word_pairs (list): List of (Hindi, English) word pairs.
        list_en_words (list): List of English words.
        list_hi_words (list): List of Hindi words.
        index (faiss.IndexFlatIP, optional): Pre-built FAISS index. Defaults to None.

    Returns:
        float: Precision@k as a percentage.
    """
    d = all_hi_matrix.shape[1]  # Dimensionality of the embeddings

    # Check if the FAISS index already exists, else build one
    if index is None:
        print("[DEBUG] FAISS index does not exist. Creating a new one.")
        index = faiss.IndexFlatIP(d)  # Inner Product (for cosine similarity) index
        faiss.normalize_L2(all_hi_matrix)  # Normalize vectors for cosine similarity
        index.add(all_hi_matrix)  # Add the Hindi embeddings to the index
        print("[DEBUG] FAISS index built and embeddings added.")

    # Filter valid pairs
    print("[DEBUG] Filtering valid pairs from test_word_pairs.")
    valid_pairs = [
        pair for pair in test_word_pairs
        if pair[1] in list_en_words and pair[0] in list_hi_words
    ]
    print(f"[DEBUG] Total valid pairs: {len(valid_pairs)}")

    # Precompute indices for valid pairs
    print("[DEBUG] Precomputing indices for valid pairs.")
    en_indices = [list_en_words.index(pair[1]) for pair in valid_pairs]
    hi_indices = [list_hi_words.index(pair[0]) for pair in valid_pairs]

    # Normalize the English embeddings once
    print("[DEBUG] Normalizing aligned English embeddings.")
    faiss.normalize_L2(en_aligned)

    # Batch query the FAISS index
    batch_size = 4  # Set an appropriate batch size
    correct_matches_at_k = 0

    print("[DEBUG] Starting batch processing for FAISS index search.")
    for start in range(0, len(en_indices), batch_size):
        end = min(start + batch_size, len(en_indices))
        batch_indices = en_indices[start:end]
        aligned_batch = en_aligned[batch_indices]  # Get batch embeddings

        print(f"[DEBUG] Processing batch: start={start}, end={end}")

        # Search for the k nearest neighbors
        _, nearest_neighbors_idx = index.search(aligned_batch, k)

        # Count correct matches
        for i, hi_idx in enumerate(hi_indices[start:end]):
            if hi_idx in nearest_neighbors_idx[i]:
                correct_matches_at_k += 1
                print(f"[DEBUG] Match found for index {hi_idx} in batch {start}-{end}")

    # Calculate precision@k
    precision_at_k = correct_matches_at_k / len(valid_pairs) * 100
    print(f"[DEBUG] Precision@k ({k}): {precision_at_k:.2f}%")
    return precision_at_k




In [ ]:


# Example usage:

precision_at_1 = calculate_precision_at_k(
    k=1,
    all_hi_matrix=all_hi_matrix,
    en_aligned=en_aligned,
    test_word_pairs=test_word_pairs,
    list_en_words=list_en_words,
    list_hi_words=list_hi_words,
)

In [ ]:
precision_at_5 = calculate_precision_at_k(
    k=5,
    all_hi_matrix=all_hi_matrix,
    en_aligned=en_aligned,
    test_word_pairs=test_word_pairs,
    list_en_words=list_en_words,
    list_hi_words=list_hi_words,
)
print(f"Precision@1: {precision_at_1:.2f}%")
print(f"Precision@5: {precision_at_5:.2f}%")

In [ ]:
import numpy as np

def calculate_top_10_for_first_examples(test_word_pairs, en_aligned, all_hi_matrix, list_en_words, list_hi_words):
    """
    Calculate cosine similarity for the first 10 test word pairs and store top 10 similar items.

    Parameters:
        test_word_pairs (list): List of (Hindi, English) word pairs.
        en_aligned (np.ndarray): Aligned English embeddings.
        all_hi_matrix (np.ndarray): Hindi word embeddings.
        list_en_words (list): List of English words.
        list_hi_words (list): List of Hindi words.

    Returns:
        list: A list of dictionaries containing the top 10 results for each test pair.
    """
    results = []

    # Normalize the Hindi matrix once for efficiency
    all_hi_matrix_norm = all_hi_matrix / np.linalg.norm(all_hi_matrix, axis=1, keepdims=True)

    for i, (hindi_word, english_word) in enumerate(test_word_pairs[:10]):  # First 10 examples
        if english_word in list_en_words and hindi_word in list_hi_words:
            en_idx = list_en_words.index(english_word)
            aligned_embedding = en_aligned[en_idx]
            hi_idx=list_hi_words.index(hindi_word)
            hi_embedding_norm=all_hi_matrix[hi_idx]
            # Normalize the query embedding
            aligned_embedding_norm = aligned_embedding / np.linalg.norm(aligned_embedding)

            # Compute cosine similarity
            similarities = np.dot(hi_embedding_norm, aligned_embedding_norm)

            score = similarities



            results.append(score
            )

    return results


# Example usage
top_10_results = calculate_top_10_for_first_examples(
    test_word_pairs, en_aligned, all_hi_matrix, list_en_words, list_hi_words
)

# Print results for the first 10 examples
for result in top_10_results:
    print(result)


In [ ]:
discriminator_params={
    'emb_dim':300,
    'dis_layers':3,
    'dis_hid_dim':512,
    'dis_dropout':0.0,
    'dis_input_dropout':0.1
}



In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np

# Hyperparameters
embedding_dim = 300  # Dimension of word embeddings
lr = 0.001  # Learning rate
num_epochs = 10  # Number of training epochs
batch_size = 32

# Sample Data (Replace with actual embeddings)
# Source and target embeddings as NumPy arrays
source_embeddings = en_matrix
target_embeddings = hi_matrix  # 1000 target embeddings

# Convert to TensorFlow tensors
source_embeddings = tf.convert_to_tensor(source_embeddings)
target_embeddings = tf.convert_to_tensor(target_embeddings)

# Define the Discriminator
class Discriminator(Model):
    def __init__(self, params):
        super(Discriminator, self).__init__()

        self.emb_dim = params['emb_dim']
        self.dis_layers = params['dis_layers']
        self.dis_hid_dim = params['dis_hid_dim']
        self.dis_dropout = params['dis_dropout']
        self.dis_input_dropout = params['dis_input_dropout']

        self.model = tf.keras.Sequential()

        # Input dropout layer
        self.model.add(layers.Dropout(self.dis_input_dropout))

        # Add intermediate layers
        for i in range(self.dis_layers + 1):
            input_dim = self.emb_dim if i == 0 else self.dis_hid_dim
            output_dim = 1 if i == self.dis_layers else self.dis_hid_dim

            # Dense layer
            self.model.add(layers.Dense(output_dim))

            # Add activation and dropout for all but the last layer
            if i < self.dis_layers:
                self.model.add(layers.LeakyReLU(alpha=0.2))
                self.model.add(layers.Dropout(self.dis_dropout))

        # Final sigmoid activation
        self.model.add(layers.Activation('sigmoid'))

    def call(self, x):
        # Input validation
        if len(x.shape) != 2 or x.shape[1] != self.emb_dim:
            raise ValueError(f"Expected input with shape (batch_size, {self.emb_dim}), but got {x.shape}")
        return tf.reshape(self.model(x), [-1])

# # Parameters for the discriminator
# discriminator_params = {
#     'emb_dim': embedding_dim,
#     'dis_layers': 2,  # Number of hidden layers
#     'dis_hid_dim': 128,  # Hidden layer dimensions
#     'dis_dropout': 0.3,  # Dropout rate
#     'dis_input_dropout': 0.1  # Input dropout rate
# }

# Initialize trainable parameters
W = tf.Variable(tf.eye(embedding_dim), trainable=True, dtype=tf.float32)  # Mapping matrix

discriminator = Discriminator(discriminator_params)

# Optimizers
optimizer_W = tf.keras.optimizers.Adam(learning_rate=lr)
optimizer_D = tf.keras.optimizers.Adam(learning_rate=lr)

# Loss function
loss_fn = tf.keras.losses.BinaryCrossentropy()

# Training loop
for epoch in range(num_epochs):
    for i in range(0, len(source_embeddings), batch_size):
        # Get mini-batches
        source_batch = source_embeddings[i:i + batch_size]
        target_batch = target_embeddings[i:i + batch_size]

        # Train the Discriminator
        with tf.GradientTape() as tape_D:
            mapped_source = tf.matmul(source_batch, W)
            source_preds = discriminator(mapped_source)
            target_preds = discriminator(target_batch)

            source_labels = tf.ones_like(source_preds)  # Source = 1
            target_labels = tf.zeros_like(target_preds)  # Target = 0

            loss_D = loss_fn(source_labels, source_preds) + loss_fn(target_labels, target_preds)

        gradients_D = tape_D.gradient(loss_D, discriminator.trainable_variables)
        optimizer_D.apply_gradients(zip(gradients_D, discriminator.trainable_variables))

        # Train the Mapping (W)
        with tf.GradientTape() as tape_W:
            mapped_source = tf.matmul(source_batch, W)
            source_preds = discriminator(mapped_source)

            # Reverse labels for adversarial training
            target_labels = tf.zeros_like(source_preds)  # Fool discriminator

            loss_W = loss_fn(target_labels, source_preds)+loss_fn(source_labels,target_preds)

        gradients_W = tape_W.gradient(loss_W, [W])
        optimizer_W.apply_gradients(zip(gradients_W, [W]))

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss_D: {loss_D.numpy():.4f}, Loss_W: {loss_W.numpy():.4f}")

print("Training complete.")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1, Discriminator Loss: 0.8928408026695251, Adversarial Loss: 5.5262675285339355
Epoch 2, Discriminator Loss: 0.7034378051757812, Adversarial Loss: 2.702890157699585
Epoch 3, Discriminator Loss: 0.5627955794334412, Adversarial Loss: 3.695969343185425
Epoch 4, Discriminator Loss: 0.07226783782243729, Adversarial Loss: 3.616457223892212
Epoch 5, Discriminator Loss: 0.6880297660827637, Adversarial Loss: 2.566481113433838
Epoch 6, Discriminator Loss: 0.45557427406311035, Adversarial Loss: 5.310499668121338
Epoch 7, Discriminator Loss: 0.4930347502231598, Adversarial Loss: 8.602936744689941
Epoch 8, Discriminator Loss: 0.8419936299324036, Adversarial Loss: 1.2795430421829224
Epoch 9, Discriminator Loss: 0.525246798992157, Adversarial Loss: 4.500785827636719
Epoch 10, Discriminator Loss: 0.13758067786693573, Adversarial Loss: 6.328951358795166


In [ ]:
np.save('/content/drive/MyDrive/sarvam_project/W.npy',W.numpy())
all_en_matrix=np.load('/content/drive/MyDrive/sarvam_project/all_en_matrix.npy')



In [ ]:
W=np.load('/content/drive/MyDrive/sarvam_project/W.npy')
en_matrix=np.load('/content/drive/MyDrive/sarvam_project/en_embeddings.npy')

In [ ]:
en_aligned_unsupervised_test=np.dot(W,en_matrix.T)
en_aligned_unsupervised_test=en_aligned_unsupervised_test.T


In [ ]:
import gc
del en_matrix
del hi_matrix
del all_en_matrix
del W
gc.collect()

In [ ]:
all_hi_matrix.shape